In [1]:
import pandas as pd
import numpy as np

In [2]:
# providing the dataset path
data1 = "https://raw.githubusercontent.com/naeljb/uthabiti/main/baseline_dataset.csv?token=GHSAT0AAAAAAB45INMQP4UL7MEPA2CSLJFEY73I2MA"
# reading the csv file and named as df
df = pd.read_csv(data1, encoding='latin-1')    

In [3]:
# checking  and # getting all the column names of the dataframe
columns_names = list(df.columns)
# print(columns_names)    # for printing the column name, remove # from this code line      

# I. Variables/columns creation 


##  1) Household size 

In [4]:
hh_size = df['m total'] + df['f total'] 

 ## 2)  Gendered household type

In [5]:
df['num_male_adult'] = df['m 18-29 years'] + df['m 30-59 years'] + df['m 60 years and above'] 
df['num_female_adult'] = df['f 18-29 years'] + df['f 30-59 years'] + df['f 60 years and above']
df['num_children'] = df['m 0-17 years'] + df['f 0-17 years']

In [6]:
# creation of a "boolean"  columne for each gendered household type 
df['FM_gender'] = df.apply(lambda row: 1 if (row['num_male_adult'] >= 1 and row['num_female_adult'] >= 1) else 0, axis=1)
df['FNM_gender'] =df.apply(lambda row: 1 if (row['num_male_adult'] == 0 and row['num_female_adult'] >= 1) else 0, axis=1)
df['MNF_gender'] = df.apply(lambda row: 1 if (row['num_male_adult'] >= 1 and row['num_female_adult'] == 0) else 0, axis=1)
df['CNA_gender'] = df.apply(lambda row: 1 if (row['num_male_adult'] == 0 and row['num_female_adult'] == 0) else 0, axis=1)

In [7]:
# Count of number of Female & Male adult households
count_FM = df['FM_gender'].value_counts()[1]
print(count_FM)

548


In [8]:
# Count of number of Female No Male adult households
count_FNM = df['FNM_gender'].value_counts()[1]
print(count_FNM)

167


In [9]:
# Count of number of Male No Female adult households
count_MNF = df['MNF_gender'].value_counts()[1]
print(count_MNF)

60


In [10]:
#Count of number of Children No adult households
count_CNA = df['CNA_gender'].value_counts()[1]
print(count_CNA)

49


In [11]:
# Creation of single column for gendered household type 
df['gender_HH_type'] = np.where((df['FM_gender'] == 1) & (df['FNM_gender'] == 0) & (df['MNF_gender'] == 0) & (df['CNA_gender'] == 0), 'F&M',
                   np.where((df['FM_gender'] == 0) & (df['FNM_gender'] == 1) & (df['MNF_gender'] ==0) & (df['CNA_gender'] == 0), 'FNM',
                   np.where((df['FM_gender'] == 0) & (df['FNM_gender'] == 0) & (df['MNF_gender'] == 1) & (df['CNA_gender'] == 0), 'MNF', 'CNA')))

In [12]:
# Count of gendered household type
df['gender_HH_type'].value_counts()

F&M    548
FNM    167
MNF     60
CNA     49
Name: gender_HH_type, dtype: int64

## 3) Food Consumption Score

In [13]:
# checking the data type for Food group consumption columns 
df[['FC01','FC02','FC03','FC04','FC05','FC06','FC07','FC08','FC09']].dtypes

FC01    object
FC02    object
FC03    object
FC04    object
FC05    object
FC06    object
FC07    object
FC08    object
FC09    object
dtype: object

In [14]:
# checking the number of missing value for food group consumption columns 
FC_missing = df[['FC01','FC02','FC03','FC04','FC05','FC06','FC07','FC08','FC09']].isna().sum()
print (FC_missing)

FC01    0
FC02    0
FC03    0
FC04    0
FC05    0
FC06    0
FC07    0
FC08    0
FC09    0
dtype: int64


In [15]:
# viewing  the different values in FCO1 and so on
print (df['FC01'].unique())
print (df['FC02'].unique())

['2 days' '7 days' '3 days' '4 days' '1 day' '6 days' '5 days' '0 day']
['3 days' '6 days' '0 day' '7 days' '5 days' '4 days' '2 days' '1 day']


In [16]:
# creating/renaming food group consumption column and keeping the first value characters (ex: 2 days to 2)
df['FC01_staple'] = df['FC01'].str.slice(stop=1)
df['FC02_Pulse']  = df['FC02'].str.slice(stop=1)
df['FC03_Vegetable'] = df['FC03'].str.slice(stop=1)
df['FC04_Fruits'] = df['FC04'].str.slice(stop=1)
df['FC05_Meat_Fish'] = df['FC05'].str.slice(stop=1)
df['FC06_Milk'] = df['FC06'].str.slice(stop=1)
df['FC07_Sugar'] = df['FC07'].str.slice(stop=1)
df['FC08_Oil'] = df['FC08'].str.slice(stop=1)
df['FC09_Condiment'] = df['FC09'].str.slice(stop=1)

In [17]:
# Transforming columns as integer data type
df['FC01_staple'] = pd.to_numeric(df['FC01_staple'], errors='coerce').astype(int)
df['FC02_Pulse'] = pd.to_numeric(df['FC02_Pulse'], errors='coerce').astype(int)
df['FC03_Vegetable'] = pd.to_numeric(df['FC03_Vegetable'], errors='coerce').astype(int)
df['FC04_Fruits'] = pd.to_numeric(df['FC04_Fruits'], errors='coerce').astype(int)
df['FC05_Meat_Fish'] = pd.to_numeric(df['FC05_Meat_Fish'], errors='coerce').astype(int)
df['FC06_Milk'] = pd.to_numeric(df['FC06_Milk'], errors='coerce').astype(int)
df['FC07_Sugar'] = pd.to_numeric(df['FC07_Sugar'], errors='coerce').astype(int)
df['FC08_Oil'] = pd.to_numeric(df['FC08_Oil'], errors='coerce').astype(int)
df['FC09_Condiment'] = pd.to_numeric(df['FC09_Condiment'], errors='coerce').astype(int)

In [18]:
# Creating food group weighted colums
df['FC01_staple_W'] = df['FC01_staple']*2
df['FC02_Pulse_W']  = df['FC02_Pulse']*3
df['FC03_Vegetable_1'] = df['FC03_Vegetable']*1
df['FC04_Fruits_W'] = df['FC04_Fruits']*1
df['FC05_Meat_Fish_W'] = df['FC05_Meat_Fish']*4
df['FC06_Milk_W'] = df['FC06_Milk']*4
df['FC07_Sugar_W'] = df['FC07_Sugar']*0.5
df['FC08_Oil_W'] = df['FC08_Oil']*0.5
df['FC09_Condiment_W'] = df['FC09_Condiment']*0

In [19]:
# computing the food consumption score
df['FCS'] = df['FC01_staple_W']+df['FC02_Pulse_W']+df['FC03_Vegetable_1']+df['FC04_Fruits_W']+df['FC05_Meat_Fish_W']+df['FC06_Milk_W']+df['FC07_Sugar_W']+df['FC08_Oil_W']+df['FC09_Condiment_W']

In [20]:
Average_FCS = df['FCS'].mean()
Average_FCS

41.98543689320388

## 4) Reduced Coping Strategy index

In [21]:
# checking the data type for coping strategy columns 
df[['CS01','CS02','CS03','CS04','CS05']].dtypes

CS01    object
CS02    object
CS03    object
CS04    object
CS05    object
dtype: object

In [22]:
# checking the number of missing value for food group consumption columns 
CS_missing = df[['CS01','CS02','CS03','CS04','CS05']].isna().sum()
print (CS_missing)

CS01    0
CS02    0
CS03    0
CS04    0
CS05    0
dtype: int64


In [23]:
# viewing  the different values in CSO1 and so on
print (df['CS01'].unique())
print (df['CS02'].unique())

['4 days' '0 day' '1 day' '2 days' '3 days' '5 days' '7 days' '6 days']
['2 days' '1 day' '0 day' '3 days' '4 days' '5 days' '6 days' '7 days']


In [24]:
# creating/renaming the coping strategy column and keeping the first value characters (ex: 2 days to 2)
df["c1"] = df ["CS01"].str[0]
df["c2"] = df ["CS02"].str[0]
df["c3"] = df ["CS03"].str[0]
df["c4"] = df ["CS04"].str[0]
df["c5"] = df ["CS05"].str[0]

In [25]:
# Transforming columns as integer data type
df["c1"] = df ["c1"].astype(int)
df["c2"] = df ["c2"].astype(int)
df["c3"] = df ["c3"].astype(int)
df["c4"] = df ["c4"].astype(int)
df["c5"] = df ["c5"].astype(int)

In [26]:
# Creating coping weighted colums
df["c1_w"] = df ["c1"]* 1
df["c2_w"] = df ["c2"]*2
df["c3_w"] = df ["c3"]*1
df["c4_w"] = df ["c4"]*3
df["c5_w"] = df ["c5"]*1

In [27]:
# Computing reduced coping strategy index
df["rcsi"] = df["c1_w"] + df["c2_w"] + df["c3_w"] + df["c4_w"] + df["c5_w"]

In [28]:
rcsi_mean = df['rcsi'].mean()
rcsi_mean

11.525485436893204

## 5) Income

In [29]:
df['status'].unique()

array(['National (host)', 'Refugee'], dtype=object)

In [30]:
df['status'].value_counts()

Refugee            541
National (host)    283
Name: status, dtype: int64

In [31]:
refugee_df = df[df['status']== 'Refugee']  # creating a subset for "refugee" from "status" column (data mining)
host_df = df[df['status']== 'host']  # creating a subset for "host" from "status" column (data mining)

I'm introducing this line for testing sync with Github